In [ ]:
import pandas as pd
import numpy as np

## Merging Dataframes



In [ ]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                         {'Name': 'Sally', 'Role': 'Course liasion'},
                         {'Name': 'James', 'Role': 'Grader'}])
staff_df = staff_df.set_index('Name')
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business'},
                           {'Name': 'Mike', 'School': 'Law'},
                           {'Name': 'Sally', 'School': 'Engineering'}])
student_df = student_df.set_index('Name')
print(staff_df.head())
print()
print(student_df.head())

In [ ]:
# inner, outer, left, right
pd.merge(left=staff_df, right=student_df, left_index=True, right_index=True, how='inner')

In [ ]:
staff_df.reset_index(inplace = True)
student_df.reset_index(inplace = True)

In [ ]:
pd.merge(left=staff_df, right=student_df, left_on='Name', right_on='Name', how='inner')

Here are two DataFrames, products and invoices. The product DataFrame has an identifier and a sticker price. The invoices DataFrame lists the people, product identifiers, and quantity. Assuming that we want to generate totals, how do we join these two DataFrames together so that we have one which lists all of the information we need?




In [ ]:
products = pd.DataFrame([{'Product ID':'4109', 'Price':5.0, 'Product':'Sushi Roll'},
                         {'Product ID':'1412', 'Price':0.5, 'Product':'Egg'},
                         {'Product ID':'8931', 'Price':1.5, 'Product':'Bagel'}])
products.set_index('Product ID', inplace = True)

In [ ]:
invoices = pd.DataFrame([{'Customer':'Ali', 'Product ID':'4109', 'Quality': 1},
                         {'Customer':'Eric', 'Product ID':'1412', 'Quality': 12},
                         {'Customer':'Ande', 'Product ID':'8931', 'Quality': 6},
                         {'Customer':'Sam', 'Product ID':'4109', 'Quality': 2}])

In [ ]:
pd.merge(products, invoices, left_index=True, right_on='Product ID', how='outer')

## Idiomatic Pandas: Making Code Pandorable

In [ ]:
df = pd.read_csv('../data/census.csv')
df.head()

Index chaining is not a good practice. Code smell.<br>
Method chaining however is effective, for example:

In [ ]:
df_new = (df.where(df['SUMLEV']==50, other=np.nan)
            .dropna()
            .set_index(['STNAME', 'CTYNAME'])
            .rename(columns={'ESTIMATESBASE2010':'Estimate_base_2010'}))

In [ ]:
df_new.sample(5)

[Extra] Making a clean df after dropping some data

In [ ]:
(df.drop(df.query('SUMLEV == 40').index)
    .set_index(['STNAME', 'CTYNAME'])
    .rename(columns={'ESTIMATESBASE2010':'Estimate_base_2010'})).sample(5)

mapping with .apply function 

`1.` Print a series with two columns that contain min and max of POPESTIMATE2010 - 2015 data by STNAME and CTNAME

In [ ]:
pop_estimate = ['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']

In [ ]:
def min_max_pop(dataframe):
    data = dataframe[pop_estimate]
    return pd.Series({'min_population': np.min(data), 'max_population': np.max(data)})

In [ ]:
df_new.apply(min_max_pop, axis=1)

`2.` Create two columns min, max of POPESTIMATE2010 - 2015 data by STNAME and CTNAME

In [ ]:
def min_max_pop2(dataframe):
    data = dataframe[pop_estimate]
    dataframe['min'] = np.min(data)
    dataframe['max'] = np.min(data)
    return dataframe

In [ ]:
df_new.apply(min_max_pop2, axis = 1).head()

`3.` Print a series of min, max of POPESTIMATE2010 - 2015 data by STNAME and CTNAME 

In [ ]:
df_new.apply(lambda x: np.max(x[pop_estimate]), axis = 1)

## Group by

In [43]:
import pandas as pd
import numpy as np

In [44]:
df = pd.read_csv('../data/census.csv')

In [45]:
df = df.where(df['SUMLEV']==50).dropna()

Get the average CENSUS2010POP by state

In [46]:
list(df.groupby('STNAME'))[0] # tuple (group, frame)

('Alabama',
     SUMLEV  REGION  DIVISION  STATE  COUNTY   STNAME            CTYNAME  \
 1     50.0     3.0       6.0    1.0     1.0  Alabama     Autauga County   
 2     50.0     3.0       6.0    1.0     3.0  Alabama     Baldwin County   
 3     50.0     3.0       6.0    1.0     5.0  Alabama     Barbour County   
 4     50.0     3.0       6.0    1.0     7.0  Alabama        Bibb County   
 5     50.0     3.0       6.0    1.0     9.0  Alabama      Blount County   
 ..     ...     ...       ...    ...     ...      ...                ...   
 63    50.0     3.0       6.0    1.0   125.0  Alabama  Tuscaloosa County   
 64    50.0     3.0       6.0    1.0   127.0  Alabama      Walker County   
 65    50.0     3.0       6.0    1.0   129.0  Alabama  Washington County   
 66    50.0     3.0       6.0    1.0   131.0  Alabama      Wilcox County   
 67    50.0     3.0       6.0    1.0   133.0  Alabama     Winston County   
 
     CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010  ...  RDOMESTICMIG2

In [47]:
%%timeit -n 10
for group, frame in df.groupby('STNAME'):
    avg = np.average(frame['CENSUS2010POP'])

11.8 ms ± 630 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [48]:
df = df.set_index('STNAME')
print(df.groupby(level=0))

def fun(string):
    if string[0] < 'M':
        return 0
    if string[0] < 'Q':
        return 1
    return 2

In [41]:
for group, frame in df.groupby(fun):
    print("There are " + str(len(frame)) + " records in group " + str(group))

There are 1177 records in group 0
There are 1134 records in group 1
There are 831 records in group 2


In [51]:
df = pd.read_csv('../data/census.csv')
df = df.where(df['SUMLEV']==50).dropna()

In [52]:
df.groupby('STNAME').agg({'CENSUS2010POP': np.average})[0:10]

,CENSUS2010POP
STNAME,
Alabama,71339.343284
Alaska,24490.724138
Arizona,426134.466667
Arkansas,38878.906667
California,642309.586207
Colorado,78581.187500
Connecticut,446762.125000
Delaware,299311.333333
District of Columbia,601723.000000


In [66]:
df.head(1)

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50.0,3.0,6.0,1.0,1.0,Alabama,Autauga County,54571.0,54571.0,54660.0,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.59227,-2.187333


In [58]:
list(df.groupby(level=0))[0]

(1,
    SUMLEV  REGION  DIVISION  STATE  COUNTY   STNAME         CTYNAME  \
 1    50.0     3.0       6.0    1.0     1.0  Alabama  Autauga County   
 
    CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010  ...  RDOMESTICMIG2011  \
 1        54571.0            54571.0          54660.0  ...          7.242091   
 
    RDOMESTICMIG2012  RDOMESTICMIG2013  RDOMESTICMIG2014  RDOMESTICMIG2015  \
 1         -2.915927         -3.012349          2.265971         -2.530799   
 
    RNETMIG2011  RNETMIG2012  RNETMIG2013  RNETMIG2014  RNETMIG2015  
 1     7.606016    -2.626146    -2.722002      2.59227    -2.187333  
 
 [1 rows x 100 columns])

In [69]:
print(type(df.groupby(level=0)[['POPESTIMATE2010','POPESTIMATE2011']]))
print(type(df.groupby(level=0)['POPESTIMATE2010']))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
<class 'pandas.core.groupby.generic.SeriesGroupBy'>


In [105]:
(df.set_index('STNAME')
 .groupby(level=0)[['POPESTIMATE2010','POPESTIMATE2011']]
 .agg({'POPESTIMATE2010': np.average, 'POPESTIMATE2011': np.sum})).iloc[0:10,:]

,POPESTIMATE2010,POPESTIMATE2011
STNAME,,
Alabama,71420.313433,4801108.0
Alaska,24621.413793,722720.0
Arizona,427213.866667,6468732.0
Arkansas,38965.253333,2938538.0
California,643691.017241,37700034.0
Colorado,78878.968750,5119480.0
Connecticut,447464.625000,3589759.0
Delaware,299930.333333,907916.0
District of Columbia,605126.000000,620472.0


In [104]:
(df.groupby('STNAME')[['POPESTIMATE2010','POPESTIMATE2011']]
 .agg({'POPESTIMATE2010': np.average, 'POPESTIMATE2011': np.sum}).iloc[0:10,:])

,POPESTIMATE2010,POPESTIMATE2011
STNAME,,
Alabama,71420.313433,4801108.0
Alaska,24621.413793,722720.0
Arizona,427213.866667,6468732.0
Arkansas,38965.253333,2938538.0
California,643691.017241,37700034.0
Colorado,78878.968750,5119480.0
Connecticut,447464.625000,3589759.0
Delaware,299930.333333,907916.0
District of Columbia,605126.000000,620472.0


## Scales

In [15]:
import pandas as pd
import numpy as np

In [22]:
grade_list = ['D-','D','D+','C-','C','C+','B-','B','B+','A-','A','A+']
evaluation_list = ['Poor','Poor','Poor','Okay','Okay','Okay','Good','Good','Good','Excellent','Excellent','Excellent']
df = pd.DataFrame(grade_list, index = evaluation_list)
df.rename(columns={0:'Grade'}, inplace=True)

In [23]:
grade_category=pd.api.types.CategoricalDtype(categories=grade_list, ordered=True)
df['Grade'] = df['Grade'].astype(grade_category)#, categories = grade_list, ordered=True)

In [24]:
df['Grade'] > 'C'

Poor         False
Poor         False
Poor         False
Okay         False
Okay         False
Okay          True
Good          True
Good          True
Good          True
Excellent     True
Excellent     True
Excellent     True
Name: Grade, dtype: bool

In [36]:
pd.get_dummies(df['Grade'] > 'B') #feature extraction using pd.get_dummies

,False,True
Poor,1,0
Poor,1,0
Poor,1,0
Okay,1,0
Okay,1,0
Okay,1,0
Good,1,0
Good,1,0
Good,0,1
Excellent,0,1


---

In [29]:
s = pd.Series(['Low', 'Low', 'High', 'Medium', 'Low', 'High', 'Low'])

In [28]:
rating_category = pd.api.types.CategoricalDtype(categories=['Low', 'Medium', 'High'], ordered=True)
s = s.astype(rating_category)

In [30]:
s = s.astype('category', categories=['Low','Medium','High'], ordered=True) # this does not work

TypeError: astype() got an unexpected keyword argument 'categories'

In [31]:
pd.get_dummies(s)

,High,Low,Medium
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,0,1,0
5,1,0,0
6,0,1,0


---

In [38]:
df = pd.read_csv('../data/census.csv')
df = df.where(df['SUMLEV']==50).dropna()

Review: different ways of Groupby

In [107]:
avgPop2010 = df.set_index('STNAME').groupby(level=0)['CENSUS2010POP'].agg('mean')

In [108]:
df.groupby('STNAME').agg({'CENSUS2010POP':['mean','sum']})

CENSUS2010POP            
                               mean         sum
STNAME                                         
Alabama                71339.343284   4779736.0
Alaska                 24490.724138    710231.0
Arizona               426134.466667   6392017.0
Arkansas               38878.906667   2915918.0
California            642309.586207  37253956.0
Colorado               78581.187500   5029196.0
Connecticut           446762.125000   3574097.0
Delaware              299311.333333    897934.0
District of Columbia  601723.000000    601723.0
Florida               280616.567164  18801310.0
Georgia                60928.635220   9687653.0
Hawaii                272060.200000   1360301.0
Idaho                  35626.863636   1567582.0
Illinois              125790.509804  12830632.0
Indiana                70476.108696   6483802.0
Iowa                   30771.262626   3046355.0
Kansas                 27172.552381   2853118.0
Kentucky               36161.391667   4339367.0
Louisiana              70833.937500   4533372.0
Maine                  83022.562500   1328361.0
Maryland              240564.666667   5773552.0
Massachusetts         467687.785714   6547629.0
Michigan              119080.000000   9883640.0
Minnesota              60964.655172   5303925.0
Mississippi            36186.548780   2967297.0
Missouri               52077.626087   5988927.0
Montana                17668.125000    989415.0
Nebraska               19638.075269   1826341.0
Nevada                158855.941176   2700551.0
New Hampshire         131647.000000   1316470.0
New Jersey            418661.619048   8791894.0
New Mexico             62399.363636   2059179.0
New York              312550.032258  19378102.0
North Carolina         95354.830000   9535483.0
North Dakota           12690.396226    672591.0
Ohio                  131096.636364  11536504.0
Oklahoma               48718.844156   3751351.0
Oregon                106418.722222   3831074.0
Pennsylvania          189587.746269  12702379.0
Rhode Island          210513.400000   1052567.0
South Carolina        100551.391304   4625364.0
South Dakota           12336.060606    814180.0
Tennessee              66801.105263   6346105.0
Texas                  98998.271654  25145561.0
Utah                   95306.379310   2763885.0
Vermont                44695.785714    625741.0
Virginia               60111.293233   7994802.0
Washington            172424.102564   6724540.0
West Virginia          33690.800000   1852994.0
Wisconsin              78985.916667   5686986.0
Wyoming                24505.478261    563626.0

In [109]:
pd.cut(x=avgPop2010, bins=10).unique()

[(11706.087, 75333.413], (390320.176, 453317.529], (579312.234, 642309.586], (75333.413, 138330.766], (264325.471, 327322.823], (201328.118, 264325.471], (453317.529, 516314.881], (138330.766, 201328.118]]
Categories (8, interval[float64]): [(11706.087, 75333.413] < (75333.413, 138330.766] < (138330.766, 201328.118] < (201328.118, 264325.471] < (264325.471, 327322.823] < (390320.176, 453317.529] < (453317.529, 516314.881] < (579312.234, 642309.586]]

Suppose we have a series that holds height data for jacket wearers. Use pd.cut to bin this data into 3 bins.

In [72]:
s = pd.Series([168, 180, 174, 190, 170, 185, 179, 181, 175, 169, 182, 177, 180, 171])

In [111]:
s = pd.cut(x=s, bins=3, labels=['Small','Medium','Large'])
s

0      Small
1     Medium
2      Small
3      Large
4      Small
5      Large
6     Medium
7     Medium
8      Small
9      Small
10    Medium
11    Medium
12    Medium
13     Small
dtype: category
Categories (3, object): [Small < Medium < Large]

## Pivot Tables

In [113]:
df = pd.read_csv('../data/cars.csv')

In [114]:
df.head(1)

,YEAR,Make,Model,Size,(kW),Unnamed: 5,TYPE,CITY (kWh/100 km),HWY (kWh/100 km),COMB (kWh/100 km),CITY (Le/100 km),HWY (Le/100 km),COMB (Le/100 km),(g/km),RATING,(km),TIME (h)
0,2012,MITSUBISHI,i-MiEV,SUBCOMPACT,49,A1,B,16.9,21.4,18.7,1.9,2.4,2.1,0,NaN,100,7


In [134]:
df.pivot_table(values='(kW)', index='YEAR', columns='Make', aggfunc='mean')

Make,BMW,CHEVROLET,FORD,KIA,MITSUBISHI,NISSAN,SMART,TESLA
YEAR,,,,,,,,
2012,NaN,NaN,NaN,NaN,49.0,80.0,NaN,NaN
2013,NaN,NaN,107.0,NaN,49.0,80.0,35.0,280.000000
2014,NaN,104.0,107.0,NaN,49.0,80.0,35.0,268.333333
2015,125.0,104.0,107.0,81.0,49.0,80.0,35.0,320.666667
2016,125.0,104.0,107.0,81.0,49.0,80.0,35.0,409.700000


In [132]:
pd.pivot_table(data=df, index=['YEAR','Make'])[0:5]

(g/km)   (kW)   (km)  CITY (Le/100 km)  CITY (kWh/100 km)  \
YEAR Make                                                                    
2012 MITSUBISHI       0   49.0  100.0               1.9               16.9   
     NISSAN           0   80.0  117.0               2.2               19.3   
2013 FORD             0  107.0  122.0               2.1               19.0   
     MITSUBISHI       0   49.0  100.0               1.9               16.9   
     NISSAN           0   80.0  117.0               2.2               19.3   

                 COMB (Le/100 km)  COMB (kWh/100 km)  HWY (Le/100 km)  \
YEAR Make                                                               
2012 MITSUBISHI               2.1               18.7              2.4   
     NISSAN                   2.4               21.1              2.6   
2013 FORD                     2.2               20.0              2.4   
     MITSUBISHI               2.1               18.7              2.4   
     NISSAN                   2.4               21.1              2.6   

                 HWY (kWh/100 km)  RATING  TIME (h)  
YEAR Make                                            
2012 MITSUBISHI              21.4     NaN       7.0  
     NISSAN                  23.0     NaN       7.0  
2013 FORD                    21.1     NaN       4.0  
     MITSUBISHI              21.4     NaN       7.0  
     NISSAN                  23.0     NaN       7.0